Import packages needed

In [1]:
import suite2p
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import glob
from natsort import natsorted
from multiprocessing import Pool
from joblib import Parallel, delayed
from pathlib import Path
import gc
gc.collect()
import importlib
import sys
# Add the directory containing your script to the Python path
sys.path.append(r'C:\Users\User\Project-SLMonlineControl\PYsubfun')

# Now you can import your script as a module
import FastBin_Suite2p as FBS

In [6]:
importlib.reload(FBS)
# Pre-set parameters, and load the only bin files in the pre-set path folde
ConfigFolder=r'C:\Users\User\Project-SLMonlineControl\config'
ops0=FBS.configLoad(ConfigFolder,'SLMsetting.yml','ops3PlanesAi203.npy')

SaveFolder=FBS.suite2pInitiate(ops0)
binFile = glob.glob(ops0['save_path0'] + '/*-001.bin')    ##<<<<<<<<<<<<<<<<<<<<<<<<<< Noted that there supposed be only one Bin Data file
##Load Data
rawBin, FramePerPlane, TotalFrameNeed=FBS.LoadBin(binFile,ops0)

Processed data would be saved inF:\LuSLMOnlineTest\SL0242-Ai203\09302024\suite2p
Folder already exists.
Total Frames 30600 is found


Suite2p processing with no motion correction; Noted following defining a function to process a plane of data; Next session, parallel processing for each plane is excuted. Much time is saved. 
Alternatively, run this process with Step2-OnlineBinSuite2p.ipynb

In [4]:
rawBin.shape

(13500, 512, 512)

In [7]:
# Define the number of parallel processes; 
# Noted that current binning movie in chuns of length 06; increase this parameter could decrease the number of frames within a chunks, using less memory
num_processes=1              ## <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
# Create a list of plane indices
plane_indices = range(0,ops0['nplanes'])
print(plane_indices)
ops1=ops0
Parallel(n_jobs=num_processes)(delayed(FBS.process_plane)(plane_idx, rawBin, SaveFolder, ops1) for plane_idx in plane_indices)

# Combine multiple plane data
#FBS.CombinePlanes(SaveFolder,ops0)

range(0, 3)
Processing plane 0
Saving folder already exists.
Binning movie in chunks of length 06
Binned movie of size [1700,512,512] created in 10.67 sec.
Binned movie denoised (for cell detection only) in 31.12 sec.
NOTE: estimated spatial scale ~6 pixels, time epochs 1.42, threshold 10.62 
0 ROIs, score=120.23
Detected 95 ROIs, 27.77 sec
After removing overlaps, 90 ROIs remain
added enhanced mean image
Masks created, 0.32 sec.
Extracted fluorescence from 90 ROIs in 10200 frames, 12.31 sec.
Extracted fluorescence from 90 ROIs in 10200 frames, 12.60 sec.
['skew', 'compact', 'npix_norm']
Processing plane 1
Saving folder already exists.
Binning movie in chunks of length 06
Binned movie of size [1700,512,512] created in 10.85 sec.
Binned movie denoised (for cell detection only) in 32.95 sec.
NOTE: estimated spatial scale ~6 pixels, time epochs 1.42, threshold 10.62 
0 ROIs, score=123.63
Detected 110 ROIs, 28.11 sec
After removing overlaps, 102 ROIs remain
added enhanced mean image
Masks 

[None, None, None]

In [8]:
FBS.CombinePlanes(SaveFolder,ops0)

appended plane 0 to combined view
appended plane 1 to combined view
appended plane 2 to combined view
3  planes combined
Folder already exists.


Run Suite2p gui from command line, load combined data above, do necessary mannual correction. Saved the data.

Once above mannual correction is done; update the result in each plane

In [9]:
FBS.PostMannual(SaveFolder,ops0)

248 of units including 44 identified cells in combined planes
